In [284]:
import os
from bs4 import BeautifulSoup as bs
import urllib2
import numpy as np
import netCDF4 as nc

from IPython.display import display, clear_output
import sys

In [264]:
#want to start at the top-left of the map
lat = np.arange(-89.5,89.5+1,1)[::-1]
lon = np.arange(-179.5,179.5+1,1)

savedir = 'W:\\2 Clients\\European Commission\\LUC4C\\Implementation\\WP7\\LUC_albedo_cf\\Kt'

In [265]:
use = os.path.join(savedir, 'Kt.nc')

tosave = nc.Dataset(use, 'w')
tosave.createDimension('latitude', 180)
tosave.createDimension('longitude', 360)
tosave.createDimension('time', 13)
tosave.createVariable('latitude', 'f4', ('latitude',))
tosave.createVariable('longitude','f4', ('longitude',))
tosave.createVariable('kt_mean', 'f4', ('latitude', 'longitude', 'time'))
tosave.createVariable('kt_min', 'f4', ('latitude', 'longitude', 'time'))
tosave.createVariable('kt_max', 'f4', ('latitude', 'longitude', 'time'))

<type 'netCDF4._netCDF4.Variable'>
float32 kt_max(latitude, longitude, time)
unlimited dimensions: 
current shape = (180, 360, 13)
filling on, default _FillValue of 9.96920996839e+36 used

In [266]:
tosave.variables['latitude'][:] = lat
tosave.variables['longitude'][:] = lon

tosave.description = """Clearness index average/min/max scraped
                        from//eosweb.larc.nasa.gov/cgi-bin/sse/grid.cgi
                        time dimension is 12 months + year average"""

In [286]:
def printer(line):
    clear_output(wait=True)
    print(line)
    sys.stdout.flush()

In [287]:
#sequence is average, min, max; 13 values each. want to assign to separate arrays

isdone = 0
counter = 0 
for i,thelat in enumerate(lat):
    for j,thelon in enumerate(lon):
        url = 'https://eosweb.larc.nasa.gov/cgi-bin/sse/grid.cgi?&num=184094&lat={}&submit=Submit&hgt=100&veg=17&sitelev=&email=skip@larc.nasa.gov&p=grid_id&p=avg_kt&step=2&lon={}'.format(thelat,thelon)
        page = urllib2.urlopen(url).read()
        soup = bs(page)
        printer(url)
        for each in soup.find_all('td', {'align': 'center'}):
            try:
                if each.text == 'n/a':
                    val = np.nan
                else:
                    val = float(each.text)
                #expect 13 vals - 12 months + yearaverage
                if counter <=13-1:  #substract 1, counter starts at zero due to zero indexing
                    tosave.variables['kt_mean'][i,j,counter%13] = val
                elif 13-1 < counter <= 26-1:
                    tosave.variables['kt_min'][i,j,counter%13] = val
                elif 26-1 < counter <= 39-1:
                    tosave.variables['kt_max'][i,j,counter%13] = val
                counter = counter + 1 # count until you get 12 months + yearavg, then reset
            except Exception as ex:
                pass
        counter = 0
        isdone = isdone + 1
        
        if isdone % 40 == 0:
            tosave.close()
            printer('Saved for {}th time. Lat is {}, Lon in {}.'.format((isdone)/40.,thelat, thelon))
            tosave = nc.Dataset(use, 'r+')

tosave.close()

print 'dunzo'

Saved for 1th time. Lat is -89.5, Lon in 179.5.
dunzo
